# Scraping hospitals in Illinois area

In [22]:
from requests import get
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup

In [47]:
ppe_illinois_url = 'https://getusppe.org/portfolio/illinois/'
ppe_illinois_url2 = 'https://findthemasks.com/give.html?state=il'

In [48]:
req = Request(ppe_illinois_url2, headers={'User-Agent': 'Mozilla/5.0'})

In [49]:
webpage = urlopen(req).read()

In [50]:
page_soup = BeautifulSoup(webpage, "html.parser")

## ACTUALLY

After doing some digging around into why the div tag for `locations-list` wasn't showing it's contents, I found it's because the data table shown is coming from an API call and the "contents" don't actually exist in the page. Digging in even more I found that the data is actually one giant JSON blob at https://findthemasks.com/data.json

In [106]:
import json

In [107]:
url = 'https://findthemasks.com/data.json'

In [108]:
blob = get(url)

In [120]:
# this will get you the response from the requests.get into json form
resp = eval(blob.text)

In [134]:
import pandas as pd

In [135]:
df = pd.DataFrame(resp)

In [150]:
df.head()

,range,majorDimension,values
0,'Form Responses 1'!A1:Y2502,ROWS,"[Approved, Deduped, Mod Status, Timestamp, Wha..."
1,'Form Responses 1'!A1:Y2502,ROWS,"[approved, deduped, residence, timestamp, name..."
2,'Form Responses 1'!A1:Y2502,ROWS,"[x, x, , 3/19/2020 11:14:25, Swedish Ballard, ..."
3,'Form Responses 1'!A1:Y2502,ROWS,"[x, x, , 3/19/2020 14:37:04, Zuckerberg San Fr..."
4,'Form Responses 1'!A1:Y2502,ROWS,"[x, x, , 3/19/2020 15:11:30, Franciscan Women'..."


It's the `values` key in the resp that has what we need

In [153]:
df2 = pd.DataFrame(df['values'])

In [235]:
import numpy as np
idx = np.random.choice(range(len(df.values)))
#df2['values'][1]  # contains headers
df2['values'][2]  # 2 and onwards contains the actual information you need

['x',
 'x',
 '',
 '3/19/2020 11:14:25',
 'Swedish Ballard',
 '5300 Tallman Ave NW\nSeattle, WA 98107',
 '5300 Tallman Ave NW\nSeattle, WA 98107',
 'Seattle',
 'WA',
 'Put in donations bin at registration desk or at medical treatment center.',
 'N95s, Surgical Masks',
 'Yes',
 '47.6674625',
 '-122.3795306',
 3]

In [176]:
headers = df2['values'][1]

In [177]:
headers

['approved',
 'deduped',
 'residence',
 'timestamp',
 'name',
 'address',
 'orig_address',
 'city',
 'state',
 'instructions',
 'accepting',
 'open_box',
 'lat',
 'lng',
 'row']

In [257]:
themast_dict = dict
themasks_dict = {k: [] for k in headers}

In [258]:
themasks_dict

{'approved': [],
 'deduped': [],
 'residence': [],
 'timestamp': [],
 'name': [],
 'address': [],
 'orig_address': [],
 'city': [],
 'state': [],
 'instructions': [],
 'accepting': [],
 'open_box': [],
 'lat': [],
 'lng': [],
 'row': []}

In [239]:
df3 = pd.DataFrame(df2['values'].iloc[2:]).reset_index(drop=True)

In [254]:

df3_test = pd.DataFrame(df3.head(30))

In [255]:
for idx, row in df3_test.iterrows():
    print(idx, row[0][9])

0 Put in donations bin at registration desk or at medical treatment center.
1 For now, call ahead: call the switchboard (628-206-8000), ask for "Donations Medical Equipment" or the front desk to arrange drop-off at the main entrance off 23rd St.
2 Bring up stairs to the Women's care desk or call and a staff member will come down to get them.
3 TBD
4 Mail to: Alexa Parker, Emergency Dept.- NNMC, 2375 E Prater Way, Sparks, NV, 89431
5 Call 580-379-5000 and ask for Mary Jencks
6 Drop off at Erie County Medical Center
Attn: Karen Taylor PA-C, UBortho #2
7 TBD
8 Email smahone4@bidmc.harvard.edu with detailed information including the brand and the quantity of the items you wish to donate. Visit bilh.org/donations for more information.
9 First Floor Main Entrance off of Highway 99
10 Please leave outside the main entrance and tell security to deliver to the ER
11 Please deliver donations to ER security desk
12 Leave at front desk
13 TBD
14 Attn: Spencer Wong, Family Medicine, Clinic 2
15 Dro

In [259]:
for idx, roww in df3.iterrows():
    themasks_dict['approved'].append(roww[0][0])
    themasks_dict['deduped'].append(roww[0][1])
    themasks_dict['residence'].append(roww[0][2])
    themasks_dict['timestamp'].append(roww[0][3])
    themasks_dict['name'].append(roww[0][4])
    themasks_dict['address'].append(roww[0][5])
    themasks_dict['orig_address'].append(roww[0][6])
    themasks_dict['city'].append(roww[0][7])
    themasks_dict['state'].append(roww[0][8])
    themasks_dict['instructions'].append(roww[0][9])
    themasks_dict['accepting'].append(roww[0][10])
    themasks_dict['open_box'].append(roww[0][11])
    themasks_dict['lat'].append(roww[0][12])
    themasks_dict['lng'].append(roww[0][13])
    #themasks_dict['row'].append(roww[0][14])

In [264]:
for key in themasks_dict.keys():
    print(key, len(themasks_dict[key]))

approved 1981
deduped 1981
residence 1981
timestamp 1981
name 1981
address 1981
orig_address 1981
city 1981
state 1981
instructions 1981
accepting 1981
open_box 1981
lat 1981
lng 1981
row 0


In [265]:
del themasks_dict['row']

In [267]:
getthemasks_df = pd.DataFrame(themasks_dict)

In [269]:
getthemasks_df.to_csv('./getthemasks_requests.csv', index=False)

In [270]:
getthemasks_df.to_csv('./piped_getthemasks_requests.csv', index=False, sep='|')

In [271]:
df_test = pd.read_csv('piped_getthemasks_requests.csv', sep='|')

In [272]:
df_test

,approved,deduped,residence,timestamp,name,address,orig_address,city,state,instructions,accepting,open_box,lat,lng
0,x,x,NaN,3/19/2020 11:14:25,Swedish Ballard,"5300 Tallman Ave NW\nSeattle, WA 98107","5300 Tallman Ave NW\nSeattle, WA 98107",Seattle,WA,Put in donations bin at registration desk or a...,"N95s, Surgical Masks",Yes,47.667462,-122.379531
1,x,x,NaN,3/19/2020 14:37:04,Zuckerberg San Francisco General Hospital,"1001 Potrero Ave\nSan Francisco, CA 94110","1001 Potrero Ave\nSan Francisco, CA 94110",San Francisco,CA,"For now, call ahead: call the switchboard (628...","N95s, Surgical Masks",No,37.755727,-122.404738
2,x,x,NaN,3/19/2020 15:11:30,Franciscan Women's Health Associates - Burien,"16045 1st Ave S\nBurien, WA 98148","16045 1st Ave S\nBurien, WA 98148",Burien,WA,Bring up stairs to the Women's care desk or ca...,"N95s, Surgical Masks",Yes,47.458564,-122.333950
3,x,x,NaN,3/19/2020 15:19:46,Moab Regional Hospital,"450 Williams Way\nMoab, UT 84532","450 Williams Way\nMoab, UT 84532",Moab,UT,TBD,"N95s, Surgical Masks, Safety Goggles",Yes,38.575156,-109.559775
4,x,x,NaN,3/19/2020 15:52:02,Northern Nevada Medical Center,"2375 E. Prater Way\nSparks, NV 89431","2375 E. Prater Way\nSparks, NV 89431",Sparks,NV,"Mail to: Alexa Parker, Emergency Dept.- NNMC, ...","N95s, Surgical Masks, Disposable Booties, Safe...",Yes,39.541116,-119.697360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1976,x,NaN,NaN,3/27/2020 21:36:44,PARC Joliet Nursing,"222 N Hammes Ave\nJoliet, IL 60435",222 north hammes,Joliet,IL,Front desk,"N95s, Face shields, Disposable booties, Safety...",Yes,41.527303,-88.131198
1977,x,NaN,Q,3/27/2020 22:23:19,North Vista Hospital,"4011 E Lake Mead Blvd\nLas Vegas, NV 89115",4011 E Lake Mead,Las vegas,NV,Please call above number ahead of time 702-929...,"N95s, Surgical masks, Safety goggles, Gowns, H...",Yes,36.195339,-115.086765
1978,x,NaN,Q,3/27/2020 22:46:16,"The Allergy Center, PA","2395 Hemby Ln\nGreenville, NC 27834",2395 Hemby Ln,Greenville,NC,2395 Hemby Ln Greenville NC 27834,"N95s, Surgical masks",Yes,35.602485,-77.408408
1979,x,NaN,Q,3/27/2020 22:39:49,Lankenau medical center,"100 E Lancaster Ave\nWynnewood, PA 19096",100 east lancaster ave,Wynnwood,PA,covidsupplydonations@mlhs.org,"N95s, Surgical masks, Face shields, Safety gog...",Yes,39.988003,-75.263492


### Trying out the dynamic splitting

In [3]:
import pandas as pd
import numpy as np
from requests import get

In [2]:
url = 'https://findthemasks.com/data.json'

In [4]:
blob = get(url)

response = eval(blob.text)

In [17]:
raw_values_df = pd.DataFrame(response['values'])

In [18]:
raw_values_df = raw_values_df.iloc[1:].reset_index(drop=True)

In [19]:
raw_values_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,approved,reason,deduped,mod_status,timestamp,name,address,orig_address,city,state,instructions,accepting,open_box,request_type,lat,lng,date,row
1,x,,x,FM,3/19/2020 11:14:25,Swedish Ballard,"5300 Tallman Ave NW\nSeattle, WA 98107","5300 Tallman Ave NW\nSeattle, WA 98107",Seattle,WA,Put in donations bin at registration desk or a...,"N95s, Surgical Masks",Yes,,47.6674625,-122.3795306,3/19,3
2,x,,x,FM,3/19/2020 14:37:04,Zuckerberg San Francisco General Hospital,"1001 Potrero Ave\nSan Francisco, CA 94110","1001 Potrero Ave\nSan Francisco, CA 94110",San Francisco,CA,"For now, call ahead: call the switchboard (628...","N95s, Surgical Masks",No,,37.7557265,-122.4047381,3/19,4
3,x,,x,FM,3/19/2020 15:11:30,Franciscan Women's Health Associates - Burien,"16045 1st Ave S\nBurien, WA 98148","16045 1st Ave S\nBurien, WA 98148",Burien,WA,Bring up stairs to the Women's care desk or ca...,"N95s, Surgical Masks",Yes,,47.4585642,-122.3339504,3/19,7
4,x,,x,FM,3/19/2020 15:19:46,Moab Regional Hospital,"450 Williams Way\nMoab, UT 84532","450 Williams Way\nMoab, UT 84532",Moab,UT,TBD,"N95s, Surgical Masks, Safety Goggles",Yes,,38.5751558,-109.5597751,3/19,8


In [21]:
raw_values_df.rename(columns=raw_values_df.iloc[0], inplace=True)

In [25]:
raw_values_df.drop([1], inplace=True).reset_index(drop=True)

AttributeError: 'NoneType' object has no attribute 'reset_index'

In [24]:
raw_values_df.head()

,approved,reason,deduped,mod_status,timestamp,name,address,orig_address,city,state,instructions,accepting,open_box,request_type,lat,lng,date,row
1,x,,x,FM,3/19/2020 11:14:25,Swedish Ballard,"5300 Tallman Ave NW\nSeattle, WA 98107","5300 Tallman Ave NW\nSeattle, WA 98107",Seattle,WA,Put in donations bin at registration desk or a...,"N95s, Surgical Masks",Yes,,47.6674625,-122.3795306,3/19,3
2,x,,x,FM,3/19/2020 14:37:04,Zuckerberg San Francisco General Hospital,"1001 Potrero Ave\nSan Francisco, CA 94110","1001 Potrero Ave\nSan Francisco, CA 94110",San Francisco,CA,"For now, call ahead: call the switchboard (628...","N95s, Surgical Masks",No,,37.7557265,-122.4047381,3/19,4
3,x,,x,FM,3/19/2020 15:11:30,Franciscan Women's Health Associates - Burien,"16045 1st Ave S\nBurien, WA 98148","16045 1st Ave S\nBurien, WA 98148",Burien,WA,Bring up stairs to the Women's care desk or ca...,"N95s, Surgical Masks",Yes,,47.4585642,-122.3339504,3/19,7
4,x,,x,FM,3/19/2020 15:19:46,Moab Regional Hospital,"450 Williams Way\nMoab, UT 84532","450 Williams Way\nMoab, UT 84532",Moab,UT,TBD,"N95s, Surgical Masks, Safety Goggles",Yes,,38.5751558,-109.5597751,3/19,8
5,x,,x,FM,3/19/2020 15:52:02,Northern Nevada Medical Center,"2375 E. Prater Way\nSparks, NV 89431","2375 E. Prater Way\nSparks, NV 89431",Sparks,NV,"Mail to: Alexa Parker, Emergency Dept.- NNMC, ...","N95s, Surgical Masks, Disposable Booties, Safe...",Yes,,39.5411157,-119.6973597,3/19,10


In [26]:
from requests import get, HTTPError
from datetime import date
import pandas as pd
import civis
import os
import logging

# Tim Velasquez
# Environment requirements in ppe_requests_ETL_requirements.txt
# Script to get PPE requests from various hospitals through GetUsPPE.org
# Also found this after experimenting with scraping:
#  https://github.com/findftm/findftm

_URL = 'https://findthemasks.com/data.json'

# Set to False if you want to export locally
_TO_PLATFORM = False

# Database information for exporting to platform
_DATABASE = ''
_SCHEMA = ''
_TABLE_PREFIX = ''

# Date suffix
today = date.today()
_DATE_SUFFIX = today.strftime('%Y%m%d')

# Filepath information for exporting locally
home = os.path.expanduser("~")
_LOCAL_EXPORT_PATH = os.path.join(home, 'Downloads')
_FILENAME = f'{_TABLE_PREFIX}_{_DATE_SUFFIX}.csv'  # ex: ftm_ppe_requests_20200328

def _get_data(url):
    """Calls URL to get data initially as string and
       turns it into a Pandas dataframe
    Parameters
    ----------
    url: str URL string leading to PPE request data

    Returns
    -------
    raw_dict: dict Dictionary form of response"""

    assert isinstance(url, str)

    logging.info('Retrieving data...')
    try:

        blob = get(url)

    except HTTPError as e:

        status_code = e.response.status_code
        print(f'HTTP error: {status_code}')
        logging.error('Data retreival FAILED')
        raise

    raw_dict = eval(blob.text)
    logging.info('Data retrieval SUCCESSFUL!')
    return raw_dict


def _clean_raw_response(raw_dict):
    """Cleans up the raw response dataframe
    Parameters
    ----------
    raw_dict: dict Raw response dictionary from _get_data()

    Returns
    -------
    clean_df: pd.DataFrame cleaned usable responses"""

    assert isinstance(raw_dict, dict)
    assert 'values' in list(raw_dict.keys())  # information we care about is in 'values' key
    assert len(raw_dict['values']) > 0

    logging.info('Cleaning data...')
    values_df = pd.DataFrame(raw_dict['values'])
    values_df = values_df.iloc[1:].reset_index(drop=True)  # table comes in with two header rows so only grab one
    values_df.rename(columns=values_df.iloc[0], inplace=True)  # renames columns with first row values
    values_df.drop([0], inplace=True)  # once you rename, drop the first row

    clean_df = values_df.reset_index(drop=True, inplace=True)  # final index reset

    logging.info('Cleaning SUCCESSFUL!')

    return clean_df


def _to_redshift(clean_df):
    """Export clean_df from _clean_raw_response() to Redshift
    Parameters
    ----------
    clean_df: pd.DataFrame
    schema: str
    table: str

    Returns
    -------
    status: str Successful or not in export to redshift"""

    logging.info('Pushing to Redshift...')

    table = f'{_SCHEMA}.{_TABLE_PREFIX}_{_DATE_SUFFIX}'

    # TODO: error handling
    fut = civis.io.dataframe_to_civis(clean_df,
                                      _DATABASE,
                                      table,
                                      delimiter='|',
                                      existing_table_rows='drop')
    status = fut.result()

    logging.info(status)
    print(f'Table name: {table}')


def _to_csv(clean_df):
    logging.info('Exporting locally...')

    filepath = f'{_LOCAL_EXPORT_PATH}/{_FILENAME}'
    clean_df.to_csv(filepath,
                    index=False)

    logging.info('Local export SUCCESSFUL!')

    print(f'Export path: {filepath}')


def main():
    """Orchestrates the data refresh"""
    raw_dict = _get_data(_URL)
    clean_df = _clean_raw_response(raw_dict)
    if _TO_PLATFORM:
        _to_redshift(clean_df)
        print('PPE request refresh completed and pushed to platform')
    else:
        print('PPE request refresh exported locally')
        _to_csv(clean_df)


In [27]:
raw_dict = _get_data(_URL)

In [29]:
clean_df = _clean_raw_response(raw_dict)

In [30]:
clean_df.head()

AttributeError: 'NoneType' object has no attribute 'head'

In [44]:
values_df = pd.DataFrame(raw_dict['values'])

In [45]:
values_df = values_df.iloc[1:].reset_index(drop=True)

In [46]:
values_df.rename(columns=values_df.iloc[0], inplace=True)

In [47]:
values_df.head()

,approved,reason,deduped,mod_status,timestamp,name,address,orig_address,city,state,instructions,accepting,open_box,request_type,lat,lng,date,row
0,approved,reason,deduped,mod_status,timestamp,name,address,orig_address,city,state,instructions,accepting,open_box,request_type,lat,lng,date,row
1,x,,x,FM,3/19/2020 11:14:25,Swedish Ballard,"5300 Tallman Ave NW\nSeattle, WA 98107","5300 Tallman Ave NW\nSeattle, WA 98107",Seattle,WA,Put in donations bin at registration desk or a...,"N95s, Surgical Masks",Yes,,47.6674625,-122.3795306,3/19,3
2,x,,x,FM,3/19/2020 14:37:04,Zuckerberg San Francisco General Hospital,"1001 Potrero Ave\nSan Francisco, CA 94110","1001 Potrero Ave\nSan Francisco, CA 94110",San Francisco,CA,"For now, call ahead: call the switchboard (628...","N95s, Surgical Masks",No,,37.7557265,-122.4047381,3/19,4
3,x,,x,FM,3/19/2020 15:11:30,Franciscan Women's Health Associates - Burien,"16045 1st Ave S\nBurien, WA 98148","16045 1st Ave S\nBurien, WA 98148",Burien,WA,Bring up stairs to the Women's care desk or ca...,"N95s, Surgical Masks",Yes,,47.4585642,-122.3339504,3/19,7
4,x,,x,FM,3/19/2020 15:19:46,Moab Regional Hospital,"450 Williams Way\nMoab, UT 84532","450 Williams Way\nMoab, UT 84532",Moab,UT,TBD,"N95s, Surgical Masks, Safety Goggles",Yes,,38.5751558,-109.5597751,3/19,8


In [48]:
values_df.drop([0], inplace=True)

In [49]:
values_df.head()

,approved,reason,deduped,mod_status,timestamp,name,address,orig_address,city,state,instructions,accepting,open_box,request_type,lat,lng,date,row
1,x,,x,FM,3/19/2020 11:14:25,Swedish Ballard,"5300 Tallman Ave NW\nSeattle, WA 98107","5300 Tallman Ave NW\nSeattle, WA 98107",Seattle,WA,Put in donations bin at registration desk or a...,"N95s, Surgical Masks",Yes,,47.6674625,-122.3795306,3/19,3
2,x,,x,FM,3/19/2020 14:37:04,Zuckerberg San Francisco General Hospital,"1001 Potrero Ave\nSan Francisco, CA 94110","1001 Potrero Ave\nSan Francisco, CA 94110",San Francisco,CA,"For now, call ahead: call the switchboard (628...","N95s, Surgical Masks",No,,37.7557265,-122.4047381,3/19,4
3,x,,x,FM,3/19/2020 15:11:30,Franciscan Women's Health Associates - Burien,"16045 1st Ave S\nBurien, WA 98148","16045 1st Ave S\nBurien, WA 98148",Burien,WA,Bring up stairs to the Women's care desk or ca...,"N95s, Surgical Masks",Yes,,47.4585642,-122.3339504,3/19,7
4,x,,x,FM,3/19/2020 15:19:46,Moab Regional Hospital,"450 Williams Way\nMoab, UT 84532","450 Williams Way\nMoab, UT 84532",Moab,UT,TBD,"N95s, Surgical Masks, Safety Goggles",Yes,,38.5751558,-109.5597751,3/19,8
5,x,,x,FM,3/19/2020 15:52:02,Northern Nevada Medical Center,"2375 E. Prater Way\nSparks, NV 89431","2375 E. Prater Way\nSparks, NV 89431",Sparks,NV,"Mail to: Alexa Parker, Emergency Dept.- NNMC, ...","N95s, Surgical Masks, Disposable Booties, Safe...",Yes,,39.5411157,-119.6973597,3/19,10


In [50]:
values_df.reset_index(drop=True, inplace=True)

In [56]:
clean_df = values_df.copy(deep=True)

In [57]:
clean_df

,approved,reason,deduped,mod_status,timestamp,name,address,orig_address,city,state,instructions,accepting,open_box,request_type,lat,lng,date,row
0,x,,x,FM,3/19/2020 11:14:25,Swedish Ballard,"5300 Tallman Ave NW\nSeattle, WA 98107","5300 Tallman Ave NW\nSeattle, WA 98107",Seattle,WA,Put in donations bin at registration desk or a...,"N95s, Surgical Masks",Yes,,47.6674625,-122.3795306,3/19,3
1,x,,x,FM,3/19/2020 14:37:04,Zuckerberg San Francisco General Hospital,"1001 Potrero Ave\nSan Francisco, CA 94110","1001 Potrero Ave\nSan Francisco, CA 94110",San Francisco,CA,"For now, call ahead: call the switchboard (628...","N95s, Surgical Masks",No,,37.7557265,-122.4047381,3/19,4
2,x,,x,FM,3/19/2020 15:11:30,Franciscan Women's Health Associates - Burien,"16045 1st Ave S\nBurien, WA 98148","16045 1st Ave S\nBurien, WA 98148",Burien,WA,Bring up stairs to the Women's care desk or ca...,"N95s, Surgical Masks",Yes,,47.4585642,-122.3339504,3/19,7
3,x,,x,FM,3/19/2020 15:19:46,Moab Regional Hospital,"450 Williams Way\nMoab, UT 84532","450 Williams Way\nMoab, UT 84532",Moab,UT,TBD,"N95s, Surgical Masks, Safety Goggles",Yes,,38.5751558,-109.5597751,3/19,8
4,x,,x,FM,3/19/2020 15:52:02,Northern Nevada Medical Center,"2375 E. Prater Way\nSparks, NV 89431","2375 E. Prater Way\nSparks, NV 89431",Sparks,NV,"Mail to: Alexa Parker, Emergency Dept.- NNMC, ...","N95s, Surgical Masks, Disposable Booties, Safe...",Yes,,39.5411157,-119.6973597,3/19,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2281,x,,,F,3/30/2020 20:10:39,Coney Island - Emergency department,"2305 Brigham St\nBrooklyn, NY 11229",2305 Brigham street,Brooklyn,NY,Please call at 347-922-6207,"N95s, Surgical masks, Face shields, Gowns, Cov...",Yes,Add a new donation site,40.5964297,-73.9341338,2913,None
2282,x,,,FM,3/30/2020 20:11:43,Sheridan Memorial Hospital,"1401 W 5th St\nSheridan, WY 82801",1401 West 5th Street,Sheridan,WY,Mailing address,"N95s, Surgical masks, Face shields, Safety gog...",Yes,Add a new donation site,44.808659,-106.975756,2914,None
2283,x,,,F,3/30/2020 20:16:10,St. Luke’s University Hospital Anderson Campus,"1872 St Luke's Blvd\nEaston, PA 18045",1872 St. Luke’s Boulevard,Easton,PA,Address to Emergency Department c/o Dr. Abby W...,"N95s, Face shields, Disposable booties, Safety...",Yes,Add a new donation site,40.6491651,-75.2884632,2915,None
2284,x,,,F,3/30/2020 20:21:05,The Lutheran Home,"2825 Bloomfield Rd\nCape Girardeau, MO 63703",2825 Bloomfield Rd,Cape Girardeau,MO,"2825 Bloomfield Rd, Cape Girardeau, MO 63701 5...","N95s, Disposable booties, Gloves, Gowns, Hand ...",Yes,Add a new donation site,37.2930476,-89.5620044,2916,None
